In [1]:
import numpy as np
import pandas as pd
import gc
import os
import time
import random
from tqdm.auto import tqdm

In [2]:
def visualize_df(df):
    print(df.shape)
    display(df.head())

In [3]:
#rank_start_date = '2020-09-09'
#rank_end_date = '2020-09-16'
rank_start_date = '2020-09-16'
rank_end_date = '2020-09-23'

In [4]:
# データの読み込み
dir = 'h-and-m-personalized-fashion-recommendations/'
path = '../input/' + dir 
transactions_train = pd.read_csv(path + 'transactions_train.csv')
transactions_train['customer_id'] = transactions_train['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

In [5]:
## matching
#sub1 = pd.read_csv(f'./submission_lightgbm_test.csv') 
#sub2 = pd.read_csv(f'./submission_linear_test.csv') 
#sub1['customer_id'] = sub1['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
#sub2['customer_id'] = sub2['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
#sub

In [6]:
sub0 = pd.read_csv('./submission_lgbm_test.csv').sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv('./submission_linear_test.csv').sort_values('customer_id').reset_index(drop=True)

In [7]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']

del sub1
gc.collect()
sub0.head()

,customer_id,prediction0,prediction1
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0751471001 0918525001 09...,0568601006 0568601043 0762846027 0751471001 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0673677002 0918292001 0898713001 0918522001 09...,0809238005 0915526001 0706016001 0706016001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0794321011 0805000001 0926502001 09...,0794321007 0794321011 0805000001 0809238005 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0730683001 0918522001 0720125001 0924243001 03...,0730683001 0720125001 0372860001 0751471043 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0896152002 0896152001 0896152002 07...,0730683050 0896152002 0791587015 0896152001 08...


In [8]:
def cust_blend(dt, W = [1,1,1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]

    #Create a list of all model predictions
    REC = []

    # Second Try
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())

    #Create a dictionary of items recommended.
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 12 items only
    return ' '.join(res[:12])


#sub0['prediction'] = sub0.apply(cust_blend, W = [1.0,1.0], axis=1)
sub0['prediction'] = sub0.apply(cust_blend, W = [1.0,0.1], axis=1)
sub0.head()

,customer_id,prediction0,prediction1,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0751471001 0918525001 09...,0568601006 0568601043 0762846027 0751471001 08...,0568601043 0568601006 0751471001 0918525001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0673677002 0918292001 0898713001 0918522001 09...,0809238005 0915526001 0706016001 0706016001 09...,0673677002 0918292001 0898713001 0918522001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0794321011 0805000001 0926502001 09...,0794321007 0794321011 0805000001 0809238005 09...,0794321007 0794321011 0805000001 0926502001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0730683001 0918522001 0720125001 0924243001 03...,0730683001 0720125001 0372860001 0751471043 05...,0730683001 0918522001 0720125001 0924243001 03...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0896152002 0896152001 0896152002 07...,0730683050 0896152002 0791587015 0896152001 08...,0730683050 0896152002 0896152001 0791587015 09...


In [9]:
del sub0['prediction0']
del sub0['prediction1']
gc.collect()

0

In [10]:
sub0['customer_id'] = sub0['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
sub0

,customer_id,prediction
0,6883939031699146327,0568601043 0568601006 0751471001 0918525001 09...
1,-7200416642310594310,0673677002 0918292001 0898713001 0918522001 09...
2,-6846340800584936,0794321007 0794321011 0805000001 0926502001 09...
3,-94071612138601410,0730683001 0918522001 0720125001 0924243001 03...
4,-283965518499174310,0730683050 0896152002 0896152001 0791587015 09...
...,...,...
1371975,7551062398649767985,0557599022 0611415001 0804992014 0720125039 08...
1371976,-9141402131989464905,0448509014 0762846008 0762846031 0706016001 09...
1371977,-8286316756823862684,0762846027 0706016002 0762846031 0689365050 08...
1371978,2551401172826382186,0448509014 0918292001 0909370001 0918522001 07...


### Divide data

In [11]:
valid_data = transactions_train.query(f"'{rank_start_date}' <= t_dat and t_dat < '{rank_end_date}'").reset_index(drop=True)
visualize_df(valid_data)

(240311, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2020-09-16,8055340703569116761,786022008,0.048441,2
1,2020-09-16,8055340703569116761,913272003,0.032288,2
2,2020-09-16,8055340703569116761,889669006,0.056508,2
3,2020-09-16,2563761487478961358,237347060,0.033881,1
4,2020-09-16,2563761487478961358,562245001,0.013542,1


In [12]:
valid_unq = valid_data.groupby('customer_id')['article_id'].apply(list).reset_index()
valid_unq['valid_true'] = valid_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
visualize_df(valid_unq)

(68984, 3)


,customer_id,article_id,valid_true
0,-9223100958908512198,"[785034009, 673677027, 673677024]",0785034009 0673677027 0673677024
1,-9223002534477110135,[793699001],0793699001
2,-9222810895170663723,"[892309001, 903306004]",0892309001 0903306004
3,-9221924794303263774,"[888343003, 892857002, 708138013]",0888343003 0892857002 0708138013
4,-9221811157628158522,[891663002],0891663002


In [13]:
valid_unq['valid_true']

0        0785034009 0673677027 0673677024
1                              0793699001
2                   0892309001 0903306004
3        0888343003 0892857002 0708138013
4                              0891663002
                       ...               
68979    0751994003 0699755081 0699755071
68980    0869331006 0562245099 0865929002
68981               0884319001 0884319003
68982                          0903924002
68983                          0914441003
Name: valid_true, Length: 68984, dtype: object

In [14]:
merged = pd.merge(sub0, valid_unq, on='customer_id', how='left').fillna('')
del merged['article_id']; gc.collect()

0

In [15]:
# remove customers who made no purchases 
merged = merged[merged['valid_true']!=''].reset_index(drop=True)
print(merged.shape)
merged.head()

(68984, 3)


,customer_id,prediction,valid_true
0,8215006181404469139,0918522001 0924243001 0762846008 0918292001 09...,0624486001
1,8282976502227659861,0889036004 0621381012 0640021012 0880017001 09...,0827487003
2,-7604547624187760215,0399136061 0556255001 0732842014 0732842021 07...,0757926001 0788575004 0640021019
3,-7555520646521469595,0448509014 0158340001 0918292001 0850917001 08...,0874110016
4,-7641768468830472284,0929744001 0865799005 0933989002 0456163060 08...,0903762001 0879189005 0158340001 0867966009 09...


In [16]:
merged

,customer_id,prediction,valid_true
0,8215006181404469139,0918522001 0924243001 0762846008 0918292001 09...,0624486001
1,8282976502227659861,0889036004 0621381012 0640021012 0880017001 09...,0827487003
2,-7604547624187760215,0399136061 0556255001 0732842014 0732842021 07...,0757926001 0788575004 0640021019
3,-7555520646521469595,0448509014 0158340001 0918292001 0850917001 08...,0874110016
4,-7641768468830472284,0929744001 0865799005 0933989002 0456163060 08...,0903762001 0879189005 0158340001 0867966009 09...
...,...,...,...
68979,2122442112338100006,0698387001 0573937001 0918522001 0924243001 08...,0874816003 0911870004
68980,-1670952078729494825,0372860001 0464297007 0918522001 0751471001 09...,0861803014 0849886010
68981,-8521419328249105955,0914441004 0706016001 0158340001 0915526001 06...,0396135007 0817472007 0715624050 0817472003 08...
68982,-5473509184230322687,0724906019 0372860001 0372860002 0924243001 04...,0750423010 0761269001


### Function

In [17]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306007
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py


def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    #remove this case in advance
    #if not actual:
    #    return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [18]:
tqdm.pandas()

mapk(
    merged['valid_true'].map(lambda x: x.split()), 
    merged['prediction'].map(lambda x: x.split()), 
    k=12
)


0.029575428893979076

In [19]:
# lgbm:linear, 1:0 -> 0.029585037401273842
# lgbm:linear, 0:1 -> 0.02613646255423653
# lgbm:linear, 1:1 -> 0.02865823492021606
# lgbm:linear, 1:0.5 -> 0.029143652240421175
# lgbm:linear, 1:0.2 -> 0.02947248924565819
# lgbm:linear, 1:0.1 -> 0.029575428893979076

## Expand

In [22]:
merged

,customer_id,prediction,valid_true
0,8215006181404469139,0918522001 0915526001 0898694001 0924243001 07...,0624486001
1,8282976502227659861,0889036004 0880017001 0621381012 0640021012 09...,0827487003
2,-7604547624187760215,0399136061 0556255001 0732842014 0732842021 07...,0757926001 0788575004 0640021019
3,-7555520646521469595,0448509014 0809238005 0918292001 0158340001 09...,0874110016
4,-7641768468830472284,0929744001 0456163060 0865799005 0933989002 08...,0903762001 0879189005 0158340001 0867966009 09...
...,...,...,...
68979,2122442112338100006,0573937001 0698387001 0652924004 0918522001 09...,0874816003 0911870004
68980,-1670952078729494825,0372860001 0464297007 0918522001 0751471043 03...,0861803014 0849886010
68981,-8521419328249105955,0914441004 0706016001 0158340001 0915526001 07...,0396135007 0817472007 0715624050 0817472003 08...
68982,-5473509184230322687,0372860001 0724906019 0372860002 0456163086 09...,0750423010 0761269001


In [23]:
merged1 = merged[['customer_id', 'prediction']]
merged1['prediction'] = merged1['prediction'].str.split()
merged2 = merged[['customer_id', 'valid_true']]
merged2['valid_true'] = merged2['valid_true'].str.split()

<ipython-input-23-729881cb8f98>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged1['prediction'] = merged1['prediction'].str.split()
<ipython-input-23-729881cb8f98>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged2['valid_true'] = merged2['valid_true'].str.split()


In [24]:
merged2 = merged2.explode('valid_true')
merged2['label'] = 1 
merged2 = merged2.rename(columns={'valid_true':'article_id'})
merged2['article_id'] = merged2['article_id'].astype('int32')

In [25]:
merged2.head()

,customer_id,article_id,label
0,8215006181404469139,624486001,1
1,8282976502227659861,827487003,1
2,-7604547624187760215,757926001,1
2,-7604547624187760215,788575004,1
2,-7604547624187760215,640021019,1


In [26]:
merged1['match_len'] = merged1['prediction'].apply(len)
merged1['match_len'] = merged1['match_len'].astype('int8')

In [27]:
merged1

,customer_id,prediction,match_len
0,8215006181404469139,"[0918522001, 0915526001, 0898694001, 092424300...",12
1,8282976502227659861,"[0889036004, 0880017001, 0621381012, 064002101...",12
2,-7604547624187760215,"[0399136061, 0556255001, 0732842014, 073284202...",12
3,-7555520646521469595,"[0448509014, 0809238005, 0918292001, 015834000...",12
4,-7641768468830472284,"[0929744001, 0456163060, 0865799005, 093398900...",12
...,...,...,...
68979,2122442112338100006,"[0573937001, 0698387001, 0652924004, 091852200...",12
68980,-1670952078729494825,"[0372860001, 0464297007, 0918522001, 075147104...",12
68981,-8521419328249105955,"[0914441004, 0706016001, 0158340001, 091552600...",12
68982,-5473509184230322687,"[0372860001, 0724906019, 0372860002, 045616308...",12


In [28]:
merged1['rec_list'] = merged1['prediction']
merged1['match_rank'] = merged1['rec_list'].apply(lambda x: [i for i in range(len(x))])
merged1 = merged1.drop(['prediction'], axis='columns')
merged1 = merged1.explode(['rec_list','match_rank'])
merged1 = merged1.rename(columns={'rec_list':'article_id'})
merged1['article_id'] = merged1['article_id'].fillna(0).astype('int32') # mathingで0件の場合は無理やり0埋め

In [29]:
merged1['label'] = 0
merged = pd.merge(merged1, merged2, on=['customer_id', 'article_id'], how='left')

merged['label_y'] = merged['label_y'].fillna(0)
merged['label'] = merged['label_x'] + merged['label_y']
merged = merged.drop(['label_x', 'label_y'], axis='columns')
merged['label'] = merged['label'].astype('bool')
merged.head()

,customer_id,match_len,article_id,match_rank,label
0,8215006181404469139,12,918522001,0,False
1,8215006181404469139,12,915526001,1,False
2,8215006181404469139,12,898694001,2,False
3,8215006181404469139,12,924243001,3,False
4,8215006181404469139,12,762846008,4,False


In [30]:
merged['label'].value_counts()

False    815934
True      12423
Name: label, dtype: int64

In [31]:
merged.groupby('match_rank')['label'].agg('mean')

match_rank
0     0.040327
1     0.026101
2     0.019784
3     0.017422
4     0.013878
5     0.011939
6     0.010884
7     0.009714
8     0.007937
9     0.007840
10    0.007023
11    0.006736
Name: label, dtype: float64

In [32]:
merged.groupby('match_rank')['label'].agg('mean').sum() # matcingの精度の目安

0.17958624251966068